# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f83dc5c5d30>
├── 'a' --> tensor([[-0.2316, -2.2175, -0.5511],
│                   [-2.3484,  1.3965, -0.9642]])
└── 'x' --> <FastTreeValue 0x7f83dc5c5d00>
    └── 'c' --> tensor([[ 1.2859, -0.3243,  1.1351,  0.1327],
                        [ 0.5730,  1.3117, -1.2694, -0.3451],
                        [ 0.8326,  0.7124,  0.4887, -1.5250]])

In [4]:
t.a

tensor([[-0.2316, -2.2175, -0.5511],
        [-2.3484,  1.3965, -0.9642]])

In [5]:
%timeit t.a

61.2 ns ± 0.0422 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f83dc5c5d30>
├── 'a' --> tensor([[ 0.9648,  0.0234,  1.1723],
│                   [ 0.7169, -0.7797,  0.7437]])
└── 'x' --> <FastTreeValue 0x7f83dc5c5d00>
    └── 'c' --> tensor([[ 1.2859, -0.3243,  1.1351,  0.1327],
                        [ 0.5730,  1.3117, -1.2694, -0.3451],
                        [ 0.8326,  0.7124,  0.4887, -1.5250]])

In [7]:
%timeit t.a = new_value

71.9 ns ± 0.0488 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2316, -2.2175, -0.5511],
               [-2.3484,  1.3965, -0.9642]]),
    x: Batch(
           c: tensor([[ 1.2859, -0.3243,  1.1351,  0.1327],
                      [ 0.5730,  1.3117, -1.2694, -0.3451],
                      [ 0.8326,  0.7124,  0.4887, -1.5250]]),
       ),
)

In [10]:
b.a

tensor([[-0.2316, -2.2175, -0.5511],
        [-2.3484,  1.3965, -0.9642]])

In [11]:
%timeit b.a

57.6 ns ± 0.185 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.4808,  2.0174,  1.0361],
               [-1.7073, -2.3017,  0.3576]]),
    x: Batch(
           c: tensor([[ 1.2859, -0.3243,  1.1351,  0.1327],
                      [ 0.5730,  1.3117, -1.2694, -0.3451],
                      [ 0.8326,  0.7124,  0.4887, -1.5250]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 0.258 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

915 ns ± 5.52 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 38.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 245 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8330279d60>
├── 'a' --> tensor([[[-0.2316, -2.2175, -0.5511],
│                    [-2.3484,  1.3965, -0.9642]],
│           
│                   [[-0.2316, -2.2175, -0.5511],
│                    [-2.3484,  1.3965, -0.9642]],
│           
│                   [[-0.2316, -2.2175, -0.5511],
│                    [-2.3484,  1.3965, -0.9642]],
│           
│                   [[-0.2316, -2.2175, -0.5511],
│                    [-2.3484,  1.3965, -0.9642]],
│           
│                   [[-0.2316, -2.2175, -0.5511],
│                    [-2.3484,  1.3965, -0.9642]],
│           
│                   [[-0.2316, -2.2175, -0.5511],
│                    [-2.3484,  1.3965, -0.9642]],
│           
│                   [[-0.2316, -2.2175, -0.5511],
│                    [-2.3484,  1.3965, -0.9642]],
│           
│                   [[-0.2316, -2.2175, -0.5511],
│                    [-2.3484,  1.3965, -0.9642]]])
└── 'x' --> <FastTreeValue 0x7f8330279c70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 32.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f83308beac0>
├── 'a' --> tensor([[-0.2316, -2.2175, -0.5511],
│                   [-2.3484,  1.3965, -0.9642],
│                   [-0.2316, -2.2175, -0.5511],
│                   [-2.3484,  1.3965, -0.9642],
│                   [-0.2316, -2.2175, -0.5511],
│                   [-2.3484,  1.3965, -0.9642],
│                   [-0.2316, -2.2175, -0.5511],
│                   [-2.3484,  1.3965, -0.9642],
│                   [-0.2316, -2.2175, -0.5511],
│                   [-2.3484,  1.3965, -0.9642],
│                   [-0.2316, -2.2175, -0.5511],
│                   [-2.3484,  1.3965, -0.9642],
│                   [-0.2316, -2.2175, -0.5511],
│                   [-2.3484,  1.3965, -0.9642],
│                   [-0.2316, -2.2175, -0.5511],
│                   [-2.3484,  1.3965, -0.9642]])
└── 'x' --> <FastTreeValue 0x7f8336e98ee0>
    └── 'c' --> tensor([[ 1.2859, -0.3243,  1.1351,  0.1327],
                        [ 0.5730,  1.3117, -1.2694, -0.3451],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 46.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.1 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2859, -0.3243,  1.1351,  0.1327],
                       [ 0.5730,  1.3117, -1.2694, -0.3451],
                       [ 0.8326,  0.7124,  0.4887, -1.5250]],
              
                      [[ 1.2859, -0.3243,  1.1351,  0.1327],
                       [ 0.5730,  1.3117, -1.2694, -0.3451],
                       [ 0.8326,  0.7124,  0.4887, -1.5250]],
              
                      [[ 1.2859, -0.3243,  1.1351,  0.1327],
                       [ 0.5730,  1.3117, -1.2694, -0.3451],
                       [ 0.8326,  0.7124,  0.4887, -1.5250]],
              
                      [[ 1.2859, -0.3243,  1.1351,  0.1327],
                       [ 0.5730,  1.3117, -1.2694, -0.3451],
                       [ 0.8326,  0.7124,  0.4887, -1.5250]],
              
                      [[ 1.2859, -0.3243,  1.1351,  0.1327],
                       [ 0.5730,  1.3117, -1.2694, -0.3451],
                       [ 0.8326,  0.7124,  0.4887, -1.5250]],

In [26]:
%timeit Batch.stack(batches)

337 µs ± 2.48 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2859, -0.3243,  1.1351,  0.1327],
                      [ 0.5730,  1.3117, -1.2694, -0.3451],
                      [ 0.8326,  0.7124,  0.4887, -1.5250],
                      [ 1.2859, -0.3243,  1.1351,  0.1327],
                      [ 0.5730,  1.3117, -1.2694, -0.3451],
                      [ 0.8326,  0.7124,  0.4887, -1.5250],
                      [ 1.2859, -0.3243,  1.1351,  0.1327],
                      [ 0.5730,  1.3117, -1.2694, -0.3451],
                      [ 0.8326,  0.7124,  0.4887, -1.5250],
                      [ 1.2859, -0.3243,  1.1351,  0.1327],
                      [ 0.5730,  1.3117, -1.2694, -0.3451],
                      [ 0.8326,  0.7124,  0.4887, -1.5250],
                      [ 1.2859, -0.3243,  1.1351,  0.1327],
                      [ 0.5730,  1.3117, -1.2694, -0.3451],
                      [ 0.8326,  0.7124,  0.4887, -1.5250],
                      [ 1.2859, -0.3243,  1.1351,  0.1327],
                   

In [28]:
%timeit Batch.cat(batches)

141 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

302 µs ± 768 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
